In [1]:
import numpy as np
import pandas as pd
import pandas_ta as ta
import os


import plotly.graph_objects as go

In [2]:
data_dir = os.path.join('D:', os.path.sep, 'Prg', 'Trading Bots', 'Market Dataset', 'dukascopy')
EUR_USD_1H = os.path.join(data_dir, 'EURUSD_Candlestick_1_Hour_BID_01.05.2023-18.05.2024.csv')
EUR_USD_1D = os.path.join(data_dir, 'EURUSD_Candlestick_1_D_BID_01.05.2023-18.05.2024.csv')
EUR_USD_Ticks = os.path.join(data_dir, 'EURUSD_Ticks_01.05.2023-01.05.2023.csv')

data_dir

'D:\\Prg\\Trading Bots\\Market Dataset\\dukascopy'

In [3]:
df_1H = pd.read_csv(EUR_USD_1H)
df_1D = pd.read_csv(EUR_USD_1D)
df_Tick = pd.read_csv(EUR_USD_Ticks)


In [4]:
print(df_1H.shape)
print(df_1D.shape)
print(df_Tick.shape)

(6558, 6)
(328, 6)
(78987, 5)


In [5]:
# Remove Holidays from 1H timeframe data.
df_1H['Date'] = df_1H['Gmt time'].str.split(' ', expand=True)[0]
holidays =  (df_1H[df_1H['Volume'] == 0].groupby('Date').count()['Close'] >= 24).index
df_ops =  df_1H[~ df_1H['Date'].isin(holidays)].copy()
print(df_ops.shape)
df_ops[df_ops['Volume'] == 0]

(6558, 7)


,Gmt time,Open,High,Low,Close,Volume,Date
